In [9]:
import numpy as np
import pandas as pd
from scipy.stats import poisson, skellam
from scipy.optimize import minimize
import datetime

In [10]:
%run get_odds.py
%run Helper.py

### League class with no printing for efficiency 

In [11]:
class league():
    def __init__(self, teams=None, p_alpha=None, q_alpha=None, alpha_hat=None, p_beta=None, q_beta=None, beta_hat=None,
                p_gamma=None, q_gamma=None, gamma_hat=None, w=0.9879 , w_b=0.77767, w3=0.9984992, delta=10, 
                 delta_g=713.4048222, 
                 promoted=dict({'p_alpha':44.28, 'q_alpha':53.71, 'p_beta':43.43, 'q_beta':39.29, 
                           'p_gamma':1.45*713.4048222, 'q_gamma':713.4048222}), 
                 relegated=dict({'p_alpha':53.71, 'q_alpha':44.28, 'p_beta':39.29, 'q_beta':43.43, 
                           'p_gamma':1.45*713.4048222, 'q_gamma':713.4048222}), trained_data=pd.DataFrame()):
        self.teams = teams
        if self.teams:
            self.NT = len(teams)
        self.p_alpha = p_alpha
        self.q_alpha = q_alpha
        self.alpha_hat = alpha_hat
        self.p_beta = p_beta
        self.q_beta = q_beta
        self.beta_hat = beta_hat
        self.p_gamma = p_gamma
        self.q_gamma = q_gamma
        self.gamma_hat = gamma_hat
        self.w = w
        self.w3 = w3
        self.w_b = w_b
        self.delta = delta
        self.delta_g = delta_g
        self.promoted = promoted
        self.relegatd = relegated
        self.trained_data = trained_data
    
    def initialise(self, teams):
        self.teams = teams
        self.NT = len(teams)
        self.p_alpha = np.array([self.delta]*self.NT, dtype=float)
        self.q_alpha = np.array([self.delta]*self.NT, dtype=float)
        self.alpha_hat = (self.p_alpha)/self.q_alpha
        
        self.p_beta = np.array([self.delta]*self.NT, dtype=float)
        self.q_beta = np.array([self.delta]*self.NT, dtype=float)
        self.beta_hat = self.p_beta/self.q_beta
        
        self.p_gamma = np.array([1.45*self.delta_g]*self.NT, dtype=float)
        self.q_gamma = np.array([self.delta_g]*self.NT, dtype=float)
        self.gamma_hat = (self.p_gamma-1)/self.q_gamma
    
    def train(self, data):
        data.loc[data['FTHG'] > 5, 'FTHG'] = 5
        data.loc[data['FTAG'] > 5, 'FTAG'] = 5
        
        lambdasH = []
        lambdasA = []
        # iterate through data
        for i in range(data.shape[0]):
            match = data.iloc[[i]]
            # get indices of home and away sides
            HT = int(np.arange(self.NT)[self.teams==match.iloc[0].loc['HomeTeam']])
            AT = int(np.arange(self.NT)[self.teams==match.iloc[0].loc['AwayTeam']])
            # get home and away lambda
            lambdaH = self.alpha_hat[HT]*self.beta_hat[AT]*self.gamma_hat[HT]
            lambdasH.append(lambdaH)
            
            lambdaA = self.alpha_hat[AT]*self.beta_hat[HT]
            lambdasA.append(lambdaA)
            
            X = int(match['FTHG'])
            Y = int(match['FTAG'])
            
            self.p_alpha[HT] = self.w*self.p_alpha[HT]+X
            self.q_alpha[HT] = self.w*self.q_alpha[HT]+self.beta_hat[AT]*self.gamma_hat[HT]
            self.alpha_hat[HT] = (self.p_alpha[HT]-1)/self.q_alpha[HT]

            self.p_alpha[AT] = self.w*self.p_alpha[AT]+Y
            self.q_alpha[AT] = self.w*self.q_alpha[AT]+self.beta_hat[HT]
            self.alpha_hat[AT] = (self.p_alpha[AT]-1)/self.q_alpha[AT]

            self.p_beta[HT] = self.w*self.p_beta[HT]+Y
            self.q_beta[HT] = self.w*self.q_beta[HT]+self.alpha_hat[AT]
            self.beta_hat[HT] = (self.p_beta[HT]-1)/self.q_beta[HT]

            self.p_beta[AT] = self.w*self.p_beta[AT]+X
            self.q_beta[AT] = self.w*self.q_beta[AT]+self.alpha_hat[HT]*self.gamma_hat[HT]
            self.beta_hat[AT] = (self.p_beta[AT]-1)/self.q_beta[AT]

            self.p_gamma[HT] = self.w3*self.p_gamma[HT]+X
            self.q_gamma[HT] = self.w3*self.q_gamma[HT]+self.alpha_hat[HT]*self.beta_hat[AT]
            self.gamma_hat[HT] = (self.p_gamma[HT]-1)/self.q_gamma[HT]
        
        phome = 1 - skellam.cdf(0, lambdasH, lambdasA)
        pdraw = skellam.pmf(0, lambdasH, lambdasA)
        paway = 1-phome-pdraw
        P = np.zeros((len(data), 3))
        P[:,0] = phome
        P[:,1] = pdraw
        P[:,2] = paway
        Z = FTRtoZ(data['FTR'])
        data['BS'] = BS(Z, P)
        data['LS'] = LS(Z, P)
        
        self.trained_data = self.trained_data.append(data, ignore_index=True) 

    def predict(self, HomeTeam, AwayTeam):
        HT = int(np.arange(self.NT)[self.teams==HomeTeam])
        AT = int(np.arange(self.NT)[self.teams==AwayTeam])
        LambdaH = self.alpha_hat[HT]*self.beta_hat[AT]*self.gamma_hat[HT]
        LambdaA = self.alpha_hat[AT]*self.beta_hat[HT]
        
        home_goals = np.zeros(6)
        away_goals = np.zeros(6)
        for i in range(5):
            home_goals[i] = poisson.pmf(i, LambdaH)
            away_goals[i] = poisson.pmf(i, LambdaA)
        home_goals[5] = 1-sum(home_goals)
        away_goals[5] = 1-sum(away_goals)
        scores = np.zeros((6,6))
        for i in range(6):
            for j in range(6):
                scores[i,j] = home_goals[i]*away_goals[j]
        
        phome = np.tril(scores, -1).sum()
        pdraw = sum(np.diag(scores))
        paway = np.triu(scores, 1).sum()
        
        # most likely result
        result = np.where(scores==np.max(scores))
        result = list(result)
        if len(result)>1:
            result = result[0]
        for i in range(len(result)):
            result[i] = int(result[i])
        
        return({'matrix':scores, 'outcomes':[phome, pdraw, paway], 'result':result})
    
    def new_season(self, teams_out, teams_promoted_in, teams_relegated_in=None):
        # record variables belonging to each team
        tracker=dict({'teams':self.teams, 'p_alpha':self.p_alpha, 'q_alpha':self.q_alpha, 'p_beta':self.p_beta, 
                      'q_beta':self.q_beta, 'p_gamma':self.p_gamma, 'q_gamma':self.q_gamma})
        teams_df = pd.DataFrame(tracker)
        # remove teams exiting league
        teams_out_index = []
        for i in range(len(self.teams)):
            if self.teams[i] in teams_out:
                teams_out_index.append(i)
        self.teams = np.delete(self.teams, teams_out_index)
        # add new teams to the league
        self.teams = np.append(self.teams, teams_promoted_in)
        if teams_relegated_in:
            self.teams = np.append(self.teams, teams_relegated_in)
        self.teams = np.array(sorted(self.teams))

        self.p_alpha = np.array([])
        self.q_alpha = np.array([])
        self.p_beta = np.array([])
        self.q_beta = np.array([])
        self.p_gamma = np.array([])
        self.q_gamma = np.array([])
        for i in range(self.NT):
            if self.teams[i] in list(teams_df['teams']):
                team_data = teams_df[teams_df['teams']==self.teams[i]]
                w_b = self.w_b
                w3 = self.w3
            elif self.teams[i] in list(teams_promoted_in):
                team_data = self.promoted
                w_b = 1
                w3 = 1
            elif self.teams[i] in list(teams_relegated_in):
                team_data = self.relegated
                w_b = 1
                w3 = 1
                
            self.p_alpha = np.append(self.p_alpha, w_b*float(team_data['p_alpha']))
            self.q_alpha = np.append(self.q_alpha, w_b*float(team_data['q_alpha']))
            self.p_beta = np.append(self.p_beta, w_b*float(team_data['p_beta']))
            self.q_beta = np.append(self.q_beta, w_b*float(team_data['q_beta']))
            self.p_gamma = np.append(self.p_gamma, w3* float(team_data['p_gamma']))
            self.q_gamma = np.append(self.q_gamma, w3*float(team_data['q_gamma']))
            
        self.alpha_hat = (self.p_alpha-1)/self.q_alpha
        self.beta_hat = (self.p_beta-1)/self.q_beta
        self.gamma_hat = (self.p_gamma-1)/self.q_gamma
        
    def train_all(self, league_str, league_below=None, league_above=None, SEA = list(range(1996, 2021))):
        NS = []
        NS_below = []
        NS_above = []
        for i in SEA:
            NS.append('AutoData/'+str(i)+league_str+'.csv')
            if league_below:
                NS_below.append('AutoData/'+str(i)+league_below+'.csv')
            if league_above:
                NS_above.append('AutoData/'+str(i)+str(league_above)+'.csv')
        
        data = pd.read_csv(NS[0])
        data = data[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
        teams = np.unique(data['HomeTeam'])
        
        self.teams = teams
        self.NT = len(teams)

        if league_below:
            data_below = pd.read_csv(NS_below[0])
            data_below = data_below[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
            teams_below = np.unique(data_below['HomeTeam'])

        if league_above:
            data_above = pd.read_csv(NS_above[0])
            data_above = data_above[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
            teams_above = np.unique(data_above['HomeTeam'])

        #print('Season: ' + str(SEA[0]), end="\r")
        self.initialise(teams)
        self.train(data)
        promoted_in=None
        relegated_in=None
        for i in range(1, len(NS)):
            #print('Season: ' + str(SEA[i]), end="\r")
            old_data = data
            old_teams = teams    
            data = pd.read_csv(NS[i], encoding = 'unicode_escape')
            data = data[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
            teams = np.unique(data['HomeTeam'])
            teams_out = list(set(old_teams) - set(teams))

            if league_below:
                old_data_below = data_below
                old_teams_below = teams_below
                data_below = pd.read_csv(NS_below[i], encoding = 'unicode_escape')
                data_below = data_below[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
                teams_below = np.unique(data_below['HomeTeam'])

            if league_above:
                old_data_above = data_above
                old_teams_above = teams_above
                data_above = pd.read_csv(NS_above[i], encoding = 'unicode_escape')
                data_above = data_above[['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]
                teams_above = np.unique(data_above['HomeTeam'])

            if league_below:
                promoted_in =  sorted(list(set(old_teams_below) & set(teams)))
            if league_above:
                relegated_in = sorted(list(set(old_teams_above) & set(teams)))
                
            if not (league_below or league_above):
                promoted_in =  sorted(list(set(teams) - set(old_teams)))

            #print('Teams Out:' + str(teams_out))
            #print('Promoted In:' + str(promoted_in))
            self.new_season(teams_out, promoted_in, relegated_in)
            self.train(data)
        #print('Training Complete')

## Premier League Optimisation

In [12]:
def PL_opt(x):
    PL = league(w=logit(x[0]), w_b=logit(x[1]), w3=logit(x[2]), delta_g=np.exp(x[3]), 
               promoted={'p_alpha':np.exp(x[4]), 'q_alpha':np.exp(x[5]), 
                         'p_beta':np.exp(x[6]), 'q_beta':np.exp(x[7]), 
                        'p_gamma':1.45*np.exp(x[3]), 'q_gamma':np.exp(x[3])})
    PL.train_all(league_str='E0', SEA=range(1996, 2011))
    return np.mean(PL.trained_data['BS'])

In [ ]:
tic = datetime.time()
PL_fit = minimize(PL_opt, [4.8, 1, 5, 6, 4, 4, 4, 4], method='BFGS', options={'disp': True})
toc = datetime.time()
print(f"Time elapsed: {toc-tic} seconds")

In [10]:
print(PL_fit['x'])

[4.83995972 0.73804617 5.81015711 6.59346276 4.18615942 4.45320225
 3.46348884 3.40559902]


In [13]:
PL = league(w=logit(PL_fit['x'][0]), w_b=logit(PL_fit['x'][1]), w3=logit(PL_fit['x'][2]), 
           promoted={'p_alpha':np.exp(PL_fit['x'][4]), 'q_alpha':np.exp(PL_fit['x'][5]), 
                         'p_beta':np.exp(PL_fit['x'][6]), 'q_beta':np.exp(PL_fit['x'][7]), 
                        'p_gamma':1.45*np.exp(PL_fit['x'][3]), 'q_gamma':np.exp(PL_fit['x'][3])})
PL.train_all(league_str='E0', SEA=range(1996, 2021))

season = []
for i in range(1996, 2021):
    season = np.append(season, [i]*380)

trained_data = PL.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + ' LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + ' LS: ' + str(np.mean(test_data['LS'])))

Training Set
BS: 0.5902585961400598 LS: 1.7347327140625275
Test Set
BS: 0.5765732700983871 LS: 1.7031602316437342


In [15]:
PL = league()
PL.train_all(league_str='E0', SEA=range(1996, 2021))
season = []
for i in range(1996, 2021):
    season = np.append(season, [i]*380)

trained_data = PL.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + 'LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + 'LS: ' + str(np.mean(test_data['LS'])))

Training Set
BS: 0.5903242296430481LS: 1.7348390194399064
Test Set
BS: 0.576441388219799LS: 1.7027621460358457


We conclude that the new optimised values give better training set performance but worse test set performance. Thus we stick with our initial values.

## Optimising for last 10 years

In [13]:
def PL_opt_new(x):
    # train up to 2020/2021 season
    PL = league(w=logit(x[0]), w_b=logit(x[1]), w3=logit(x[2]), delta_g=np.exp(x[3]), 
               promoted={'p_alpha':np.exp(x[4]), 'q_alpha':np.exp(x[5]), 
                         'p_beta':np.exp(x[6]), 'q_beta':np.exp(x[7]), 
                        'p_gamma':1.45*np.exp(x[3]), 'q_gamma':np.exp(x[3])})
    PL.train_all(league_str='E0', SEA=list(range(1996, 2021)))
    # account for covid, no home advantage, train in 2020/2021 season
    new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")
    PL.new_season(teams_out=['Bournemouth', 'Norwich', 'Watford'], teams_promoted_in=['Fulham','Leeds', 'West Brom'])
    PL.p_gamma = np.array([101]*20, dtype=float)
    PL.q_gamma = np.array([100]*20, dtype=float)
    PL.gamma_hat = (PL.p_gamma-1)/PL.q_gamma
    PL.train(new_data)
    PL.trained_data.loc[9500:, 'SEA'] = 2021
    # adjust for being back to normal, home team advantage is back, train 2021/2022 season
    PL.p_alpha =PL.p_alpha * 0.99
    PL.p_beta =PL.p_beta * 0.99
    PL.q_alpha =PL.q_alpha * 0.99
    PL.q_beta =PL.q_beta * 0.99
    PL.new_season(teams_out=['Fulham', 'Sheffield United', 'West Brom'], teams_promoted_in=['Brentford','Norwich', 'Watford'])
    PL.p_gamma = np.array([141]*20, dtype=float)
    PL.q_gamma = np.array([100]*20, dtype=float)
    PL.gamma_hat = (PL.p_gamma-1)/PL.q_gamma
    new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2122/E0.csv")
    PL.train(new_data)
    PL.trained_data.loc[9880:, 'SEA'] = 2022
    # train 2022/2023 season
    PL.new_season(teams_out=['Norwich', 'Watford', 'Burnley'], teams_promoted_in=['Fulham',"Nott'm Forest", 'Bournemouth'])
    new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2223/E0.csv")
    PL.train(new_data)
    PL.trained_data.loc[10260:, 'SEA'] = 2023
    PL.trained_data.tail(10)
    # train 2023/2024 season
    PL.new_season(teams_out=['Leicester', 'Southampton', 'Leeds'], teams_promoted_in=['Luton','Burnley', 'Sheffield United'])
    new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2324/E0.csv")
    PL.train(new_data)
    PL.trained_data.loc[10640:, 'SEA'] = 2024
    PL.trained_data.tail(10)
    return np.mean(PL.trained_data[PL.trained_data['SEA'].astype('float') > 2015]['BS'])

In [19]:
print(PL_fit_new['x'])

[ 3.92907402  9.22859382  2.54662282  9.46227649  2.80422804  9.34189996
  5.5578325  -0.98247362]


In [14]:
tic = datetime.datetime.now()
PL_fit_new = minimize(PL_opt_new, [4.8, 1, 5, 6, 4, 4, 4, 4], method='BFGS', options={'disp': True})
toc = datetime.datetime.now()
print(f"Time elapsed: {toc-tic} seconds")

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

...

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\F

Optimization terminated successfully.
         Current function value: 0.572566
         Iterations: 83
         Function evaluations: 855
         Gradient evaluations: 95


TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [27]:
PL = league(w=logit(PL_fit_new['x'][0]), w_b=logit(PL_fit_new['x'][1]), w3=logit(PL_fit_new['x'][2]), 
           promoted={'p_alpha':np.exp(PL_fit_new['x'][4]), 'q_alpha':np.exp(PL_fit_new['x'][5]), 
                         'p_beta':np.exp(PL_fit_new['x'][6]), 'q_beta':np.exp(PL_fit_new['x'][7]), 
                        'p_gamma':1.45*np.exp(PL_fit_new['x'][3]), 'q_gamma':np.exp(PL_fit_new['x'][3])})
PL.train_all(league_str='E0', SEA=range(1996, 2021))
# account for covid, no home advantage, train in 2020/2021 season
new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")
PL.new_season(teams_out=['Bournemouth', 'Norwich', 'Watford'], teams_promoted_in=['Fulham','Leeds', 'West Brom'])
PL.p_gamma = np.array([101]*20, dtype=float)
PL.q_gamma = np.array([100]*20, dtype=float)
PL.gamma_hat = (PL.p_gamma-1)/PL.q_gamma
PL.train(new_data)
PL.trained_data.loc[9500:, 'SEA'] = 2021
# adjust for being back to normal, home team advantage is back, train 2021/2022 season
PL.p_alpha =PL.p_alpha * 0.99
PL.p_beta =PL.p_beta * 0.99
PL.q_alpha =PL.q_alpha * 0.99
PL.q_beta =PL.q_beta * 0.99
PL.new_season(teams_out=['Fulham', 'Sheffield United', 'West Brom'], teams_promoted_in=['Brentford','Norwich', 'Watford'])
PL.p_gamma = np.array([141]*20, dtype=float)
PL.q_gamma = np.array([100]*20, dtype=float)
PL.gamma_hat = (PL.p_gamma-1)/PL.q_gamma
new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2122/E0.csv")
PL.train(new_data)
PL.trained_data.loc[9880:, 'SEA'] = 2022
# train 2022/2023 season
PL.new_season(teams_out=['Norwich', 'Watford', 'Burnley'], teams_promoted_in=['Fulham',"Nott'm Forest", 'Bournemouth'])
new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2223/E0.csv")
PL.train(new_data)
PL.trained_data.loc[10260:, 'SEA'] = 2023
PL.trained_data.tail(10)
# train 2023/2024 season
PL.new_season(teams_out=['Leicester', 'Southampton', 'Leeds'], teams_promoted_in=['Luton','Burnley', 'Sheffield United'])
new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2324/E0.csv")
PL.train(new_data)
PL.trained_data.loc[10640:, 'SEA'] = 2024
PL.trained_data.tail(10)

season = []
for i in range(1996, 2025):
    season = np.append(season, [i]*380)

trained_data = PL.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + ' LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + ' LS: ' + str(np.mean(test_data['LS'])))

C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: divide by zero encountered in log
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))
C:\Users\danie\Documents\Python Projects\Football\Helper.py:37: RuntimeWarning: invalid value encountered in multiply
  summa = summa + sum(X[i,]*np.log(P[i,])) + sum((1-X[i,])*np.log(1-P[i,]))


Training Set
BS: 0.7658877029256823 LS: 2.579054300925513
Test Set
BS: 0.584229485744481 LS: 1.725129054157838


In [34]:
PL.trained_data.groupby('season').mean()

,FTHG,FTAG,BS,LS,HTHG,HTAG,HS,AS,HST,AST,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,SEA
season,,,,,,,,,,,,,,,,,,,,,
1996.0,1.513158,1.071053,0.601531,1.763608,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997.0,1.452632,1.081579,1.017936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998.0,1.539474,1.115789,1.043401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999.0,1.447368,1.055263,0.982169,4.745993,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000.0,1.652632,1.118421,0.883813,3.884509,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001.0,1.531579,1.065789,0.891984,3.733389,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002.0,1.455263,1.165789,0.772553,3.045164,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003.0,1.497368,1.123684,0.770446,2.632658,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004.0,1.500000,1.157895,0.767289,2.475029,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Spanish League Optimisation

In [122]:
def SP_opt(x):
    SP1 = league(w=logit(x[0]), w_b=logit(x[1]), w3=logit(x[2]), delta_g=np.exp(x[3]), 
               promoted={'p_alpha':np.exp(x[4]), 'q_alpha':np.exp(x[5]), 
                         'p_beta':np.exp(x[6]), 'q_beta':np.exp(x[7]), 
                        'p_gamma':1.45*np.exp(x[3]), 'q_gamma':np.exp(x[3])})
    SP1.train_all(league_str='SP1', league_below='SP2', SEA=range(1998, 2011))
    return np.mean(SP1.trained_data['BS'])

In [123]:
SP_fit = minimize(SP_opt, [4.8, 1, 5, 6, 4, 4, 4, 4], method='BFGS', options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.607079
         Iterations: 98
         Function evaluations: 999
         Gradient evaluations: 111


In [132]:
print(logit(SP_fit['x'][:3]))
print(np.exp(SP_fit['x'][3:]))

[0.99999434 0.55454457 0.97476397]
[17089.91198933   108.41052871   120.97299094    95.10527751
    88.56909694]


In [125]:
SP1 = league(w=logit(SP_fit['x'][0]), w_b=logit(SP_fit['x'][1]), w3=logit(SP_fit['x'][2]), 
           promoted={'p_alpha':np.exp(SP_fit['x'][4]), 'q_alpha':np.exp(SP_fit['x'][5]), 
                         'p_beta':np.exp(SP_fit['x'][6]), 'q_beta':np.exp(SP_fit['x'][7]), 
                        'p_gamma':1.45*np.exp(SP_fit['x'][3]), 'q_gamma':np.exp(SP_fit['x'][3])})
SP1.train_all(league_str='SP1', league_below='SP2', SEA=range(1998, 2021))

season = []
for i in range(1998, 2021):
    season = np.append(season, [i]*380)

trained_data = SP1.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + ' LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + ' LS: ' + str(np.mean(test_data['LS'])))

Training Set
BS: 0.6071976749327025 LS: 1.7755582504286556
Test Set
BS: 0.567089513078663 LS: 1.6772783366113735


In [126]:
SP1 = league()
SP1.train_all(league_str='SP1', league_below='SP2', SEA=range(1998, 2021))

season = []
for i in range(1998, 2021):
    season = np.append(season, [i]*380)

trained_data = SP1.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + ' LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + ' LS: ' + str(np.mean(test_data['LS'])))

Training Set
BS: 0.6078238048580611 LS: 1.776746870224127
Test Set
BS: 0.5667161402063526 LS: 1.6762430050104256


Thus, we conclude that our optimisation performs worse than our previous PL values.

## German League Optimisation

In [127]:
def D1_opt(x):
    D1 = league(w=logit(x[0]), w_b=logit(x[1]), w3=logit(x[2]), delta_g=np.exp(x[3]), 
               promoted={'p_alpha':np.exp(x[4]), 'q_alpha':np.exp(x[5]), 
                         'p_beta':np.exp(x[6]), 'q_beta':np.exp(x[7]), 
                        'p_gamma':1.45*np.exp(x[3]), 'q_gamma':np.exp(x[3])})
    D1.train_all(league_str='D1', league_below='D2', SEA=range(1998, 2011))
    return np.mean(D1.trained_data['BS'])

In [128]:
D1_fit = minimize(D1_opt, [4.8, 1, 5, 6, 4, 4, 4, 4], method='BFGS', options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.603446
         Iterations: 72
         Function evaluations: 738
         Gradient evaluations: 82


In [129]:
print(D1_fit['x'])
print(logit(D1_fit['x'][:3]))
print(np.exp(D1_fit['x'][3:]))

[34.36061782  0.64673513  2.41632477 15.64962755 21.2645031  21.5181868
 22.2084481  22.08347735]
[1.         0.65627436 0.91806371]
[6.25960440e+06 1.71813133e+09 2.21426598e+09 4.41577019e+09
 3.89701747e+09]


In [130]:
D1 = league(w=logit(D1_fit['x'][0]), w_b=logit(D1_fit['x'][1]), w3=logit(D1_fit['x'][2]), 
           promoted={'p_alpha':np.exp(D1_fit['x'][4]), 'q_alpha':np.exp(D1_fit['x'][5]), 
                         'p_beta':np.exp(D1_fit['x'][6]), 'q_beta':np.exp(D1_fit['x'][7]), 
                        'p_gamma':1.45*np.exp(D1_fit['x'][3]), 'q_gamma':np.exp(D1_fit['x'][3])})
D1.train_all(league_str='D1', league_below='D2', SEA=range(1998, 2021))

season = []
for i in range(1998, 2021):
    season = np.append(season, [i]*306)

trained_data = D1.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + ' LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + ' LS: ' + str(np.mean(test_data['LS'])))

Training Set
BS: 0.6053155726194497 LS: 1.772075451036138
Test Set
BS: 0.600923066730202 LS: 1.7611932346347134


In [131]:
D1 = league()
D1.train_all(league_str='D1', league_below='D2', SEA=range(1998, 2021))

season = []
for i in range(1998, 2021):
    season = np.append(season, [i]*306)

trained_data = D1.trained_data
trained_data['season'] = season

training_data = trained_data[trained_data['season']<2011]
test_data = trained_data[trained_data['season']>=2011]

print('Training Set')
print('BS: ' + str(np.mean(training_data['BS'])) + ' LS: ' + str(np.mean(training_data['LS'])))
print('Test Set')
print('BS: ' + str(np.mean(test_data['BS'])) + ' LS: ' + str(np.mean(test_data['LS'])))

Training Set
BS: 0.6053873891505165 LS: 1.7723009313151803
Test Set
BS: 0.5984057446979939 LS: 1.7541896378788524


### This script brings us to the conclusion that our default values perform better than new optimised values